In [2]:
import pandas as pd
import spacy

In [3]:
df = pd.read_csv('train.csv')
df.head()

,id,raw_address,POI/street
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika
1,1,"aye, jati sampurna",/
2,2,setu siung 119 rt 5 1 13880 cipayung,/siung
3,3,"toko dita, kertosono",toko dita/
4,4,jl. orde baru,/jl. orde baru


In [4]:
split = df['POI/street'].str.split("/", expand=True)
df["POI"] = split[0]
df['street'] = split[1]

In [5]:
df.head()

,id,raw_address,POI/street,POI,street
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika,,jl kapuk timur delta sili iii lippo cika
1,1,"aye, jati sampurna",/,,
2,2,setu siung 119 rt 5 1 13880 cipayung,/siung,,siung
3,3,"toko dita, kertosono",toko dita/,toko dita,
4,4,jl. orde baru,/jl. orde baru,,jl. orde baru


In [6]:
train_s_c = []
train_p_c = []

for index, row in df.iterrows():
    add = row['raw_address']
    street = row['street']
    poi = row['POI']

    if street:
        s_pos = add.find(street)
        if s_pos != -1:
            entities = []
            entities.append((s_pos, s_pos + len(street) + 1, 'street'))
            train_s_c.append((add, {"entities" : entities}))

    if poi:
        p_pos = add.find(poi)
        if p_pos != -1:
            entities = []
            entities.append((p_pos, p_pos + len(poi) + 1, 'POI'))
            train_p_c.append((add, {"entities" : entities}))

In [7]:
from tqdm import tqdm
from spacy.tokens import DocBin

nlp = spacy.blank("en") # load a new spacy model


db = DocBin() # create a DocBin object
for text, annot in tqdm(train_s_c): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            pass
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)
db.to_disk("./train_s_c.spacy") # save the docbin object


db = DocBin() # create a DocBin object
for text, annot in tqdm(train_p_c): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            pass
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)
db.to_disk("./train_p_c.spacy") # save the docbin object

100%|██████████| 75351/75351 [00:24<00:00, 3090.52it/s]


In [1]:
#training model

In [7]:
nlps = spacy.load(R".\output_s\model-best")

In [15]:
test = pd.read_csv('test.csv')

In [22]:
test['street'] = ['']*50000

In [23]:
test.head()

,id,raw_address,street
0,0,s. par 53 sidanegara 4 cilacap tengah,
1,1,"angg per, baloi indah kel. lubuk baja",
2,2,"asma laun, mand imog,",
3,3,"ud agung rej, raya nga sri wedari karanganyar",
4,4,"cut mutia, 35 baiturrahman",


In [74]:
street_d = {var for var in df['street']}

In [76]:
for index, row in test.iterrows():
    test_str = ""
    addr = row['raw_address']
    doc = nlps(addr)
    if len(doc.ents) > 0:
        test_str = doc.ents[0].text
        if test_str[-1] == ',':
            test_str = test_str[:-1]
    if test_str in street_d:
        test['street'].iloc[index] = test_str

C:\Users\vuduc\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [80]:
test['street'].value_counts()

                      6444
jend sudi              251
ahmad yani             161
jend ahmad yani        108
pem                     88
                      ... 
haji goti                1
karya sari               1
agus ahmad               1
komp sek. neg cido       1
tam sega i               1
Name: street, Length: 28119, dtype: int64

In [81]:
test.iloc[-10:]

,id,raw_address,street
49990,49990,"tre jaya indon, h maul hasan,",h maul hasan
49991,49991,river gar boule cakung timur rw 7 cakung,river gar boule
49992,49992,toddo iv pandang panakkukang,toddo iv
49993,49993,gayaman nasi 24 12 mojoanyar,nasi 24
49994,49994,m. tau kandai dua woja,m. tau
49995,49995,toko mbak farid semboro semboro,
49996,49996,"vie - tk. ridho kids, vete 3 cari, 16720 ciawi",vete 3 cari
49997,49997,"mart dan roti bakar malabar, nasio,",nasio
49998,49998,graha indah pamulang jl. mujair raya bambu apu...,jl. mujair raya
49999,49999,"adi,",adi


In [93]:
poi_d = {var for var in df['POI']}

In [84]:
nlpp = spacy.load(R".\output_p\model-best")

In [97]:
test['POI'] = ['']*50000

In [98]:
for index, row in test.iterrows():
    test_str = ""
    addr = row['raw_address']
    doc = nlpp(addr)
    if len(doc.ents) > 0:
        test_str = doc.ents[0].text
        if test_str[-1] == ',':
            test_str = test_str[:-1]
    if test_str in poi_d:
        test['POI'].iloc[index] = test_str

C:\Users\vuduc\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [100]:
test['POI/street'] = ['']*50000

In [103]:
test['POI'].value_counts()

                       44896
perumahan wahana          68
palm spring regency       33
tower sakura              31
alfamart                  28
                       ...  
merpati                    1
pawang tenun               1
perumahan pesona           1
toko mega                  1
griya laundry              1
Name: POI, Length: 2489, dtype: int64

In [104]:
test['POI/street'] = test['POI'] + '/' + test['street']

In [106]:
test.iloc[100:110]

,id,raw_address,street,POI,POI/street
100,100,"tiki, bule utara kelapa gad, kelapa gading timur",bule utara kelapa gad,tiki,tiki/bule utara kelapa gad
101,101,sate 61151 manyar,sate,,/sate
102,102,jl.a dah 9 kramat pela rt 1 1 kebayoran baru,jl.a dah,,/jl.a dah
103,103,cafe fun rck,,,/
104,104,wonosari pudak 22 wonosari,wonosari pudak,,/wonosari pudak
105,105,tham 5 49 ketapang rt 3 rw 2 cipondoh,tham 5,,/tham 5
106,106,jogoka gg. sidol mantrijeron,jogoka gg. sidol,,/jogoka gg. sidol
107,107,"move on cell, pasundan no 1 75123 samarinda ulu",pasundan,,/pasundan
108,108,"lat sim, klaten utara",lat sim,,/lat sim
109,109,"sutra putih marg, depok",marg,sutra putih,sutra putih/marg


In [111]:
answer = pd.DataFrame(test['POI/street'])
answer.index.name = 'id'

In [112]:
answer.head()

,POI/street
id,
0,/s. par
1,/angg per
2,/mand imog
3,/raya nga
4,/cut mutia


In [113]:
answer.to_csv(r'C:\Users\vuduc\Desktop\submission.csv')